In [1]:
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import warnings
warnings.filterwarnings("ignore")

In [2]:
# re-size all the images to this
IMAGE_SIZE = [256, 256]

train_path = '/kaggle/input/final-year-2/Data/train_f_images'
valid_path = '/kaggle/input/final-year-2/Data/train_images'

In [3]:
from keras.applications.vgg16 import VGG16
SIZE = 224

VGG = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
VGG_fc = VGG16(weights='imagenet', include_top=True, input_shape=(SIZE, SIZE, 3))

VGG_model = Sequential()
VGG_model_fc = Sequential()

for layer in VGG.layers:
    VGG_model.add(layer) 
    
for layer in VGG_fc.layers[:-1]:
    VGG_model_fc.add(layer) 
    
# Freeze the layers    
for layer in VGG_model.layers:
    layer.trainable=False
    
for layer in VGG_model_fc.layers:
    layer.trainable=False

2022-12-04 08:37:58.221841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:37:58.384523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:37:58.385558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:37:58.387952: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

   16384/58889256 [..............................] - ETA: 0s

2022-12-04 08:37:58.394087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:37:58.394841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:37:58.395560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:38:00.663313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 08:38:00.664161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

553476096/553467096 [==============================] - 2s 0us/step


In [4]:
VGG_model_fc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [5]:
import os
import cv2
import glob
#training and labes numpy array conversion
train_images=[]
train_labels=[]

for file in os.listdir('/kaggle/input/final-year-2/Data/train_f_images/diseased'):
    if file[0] == 'd':
        train_labels.append('diseased')
    else:
        train_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/train_f_images/diseased',file))
    img=cv2.resize(img, (SIZE,SIZE))
    train_images.append(img)
    
for file in os.listdir('/kaggle/input/final-year-2/Data/train_f_images/healthy'):
    if file[0] == 'd':
        train_labels.append('diseased')
    else:
        train_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/train_f_images/healthy',file))
    img=cv2.resize(img, (SIZE,SIZE))
    train_images.append(img)

    
train_images=np.array(train_images)
train_labels=np.array(train_labels)

In [6]:
#training and labes numpy array conversion
test_images=[]
test_labels=[]

for file in os.listdir('/kaggle/input/final-year-2/Data/test_images/diseased'):
    if file[0] == 'd':
        test_labels.append('diseased')
    else:
        test_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/test_images/diseased',file))
    img=cv2.resize(img, (SIZE,SIZE))
    test_images.append(img)
    
for file in os.listdir('/kaggle/input/final-year-2/Data/test_images/healthy'):
    if file[0] == 'd':
        test_labels.append('diseased')
    else:
        test_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/test_images/healthy',file))
    img=cv2.resize(img, (SIZE,SIZE))
    test_images.append(img)

    
test_images=np.array(test_images)
test_labels=np.array(test_labels)

In [7]:
frature_extractor=VGG_model_fc.predict(train_images)

2022-12-04 08:38:13.082505: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-04 08:38:14.538379: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [8]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [9]:
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,plot_confusion_matrix
import keras.utils
from tensorflow.keras.utils import to_categorical
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(train_labels)
train_labels=label_encoder.transform(train_labels)
frature_extractor_test=VGG_model_fc.predict(test_images)
test_labels=label_encoder.transform(test_labels)

In [10]:
from sklearn.naive_bayes import GaussianNB
gb = GaussianNB()
gb.fit(frature_extractor,train_labels)
y_pred = gb.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = gb.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))
print("Precision:{}".format(precision_score(test_labels,y_pred)))
print("Recall:{}".format(recall_score(test_labels,y_pred)))
print("F1 Score:{}".format(f1_score(test_labels,y_pred)))

train accuracy:0.8506912442396314
accuracy:0.7276190476190476
Precision:0.688034188034188
Recall:0.696969696969697
F1 Score:0.6924731182795699


In [11]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(frature_extractor,train_labels)
y_pred = dc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = dc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))
print("Precision:{}".format(precision_score(test_labels,y_pred)))
print("Recall:{}".format(recall_score(test_labels,y_pred)))
print("F1 Score:{}".format(f1_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.539047619047619
Precision:0.43956043956043955
Recall:0.17316017316017315
F1 Score:0.2484472049689441


In [12]:
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(n_estimators = 50)
rc.fit(frature_extractor,train_labels)
y_pred = rc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = rc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))
print("Precision:{}".format(precision_score(test_labels,y_pred)))
print("Recall:{}".format(recall_score(test_labels,y_pred)))
print("F1 Score:{}".format(f1_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.7485714285714286
Precision:0.9380530973451328
Recall:0.4588744588744589
F1 Score:0.6162790697674418


In [13]:
from sklearn.linear_model import LogisticRegression
lc = LogisticRegression()
lc.fit(frature_extractor,train_labels)
y_pred = lc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = lc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))
print("Precision:{}".format(precision_score(test_labels,y_pred)))
print("Recall:{}".format(recall_score(test_labels,y_pred)))
print("F1 Score:{}".format(f1_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.6704761904761904
Precision:0.9027777777777778
Recall:0.2813852813852814
F1 Score:0.4290429042904291


In [14]:
import xgboost
xgb = xgboost.XGBRFClassifier()
xgb.fit(frature_extractor,train_labels)
y_pred = xgb.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = xgb.predict(frature_extractor_test)
print("test accuracy:{}".format(accuracy_score(test_labels,y_pred)))
print("Precision:{}".format(precision_score(test_labels,y_pred)))
print("Recall:{}".format(recall_score(test_labels,y_pred)))
print("F1 Score:{}".format(f1_score(test_labels,y_pred)))

train accuracy:0.9917050691244239
test accuracy:0.8342857142857143
Precision:0.8913043478260869
Recall:0.70995670995671
F1 Score:0.7903614457831326


In [15]:
from sklearn.model_selection import KFold,StratifiedKFold
kf = KFold(n_splits = 5, random_state = 1, shuffle = True)

In [16]:
preds = []
fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
i = 1
for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        rc = RandomForestClassifier()
        rc.fit(xtrain,ytrain)
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        y_pred = rc.predict(frature_extractor_test)
        preds.append(y_pred)
        i+=1
preds = np.mean(np.column_stack(preds), axis=1)
new_preds = []
for x in preds:
    if x>=0.5:
        new_preds.append(1)
    else:
        new_preds.append(0)
print("Total Test Accuracy:{}".format(accuracy_score(new_preds,test_labels)))
print("Precision:{}".format(precision_score(new_preds,test_labels)))
print("Recall:{}".format(recall_score(new_preds,test_labels)))
print("F1 Score:{}".format(f1_score(new_preds,test_labels)))

For Split 1
train accuracy:1.0
test accuracy:0.8940092165898618
For Split 2
train accuracy:1.0
test accuracy:0.9354838709677419
For Split 3
train accuracy:1.0
test accuracy:0.8617511520737328
For Split 4
train accuracy:1.0
test accuracy:0.9170506912442397
For Split 5
train accuracy:1.0
test accuracy:0.9262672811059908
Total Test Accuracy:0.7657142857142857
Precision:0.4805194805194805
Recall:0.9736842105263158
F1 Score:0.6434782608695653


In [17]:
models = 1
import numpy as np
import pandas as pd 
import optuna 
import sklearn
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history,plot_param_importances

In [18]:

def objective(trial):
    #empty list for training and testing roc_scores
#     print("model number : {}".format(models))
#     models+=1
    test_score = []
    
    #optunas parameters suggestion

    param_grid = {
              'n_estimators': trial.suggest_int('n_estimators', 1,101),
                'max_depth': trial.suggest_int('max_depth',1,6),
              'random_state': 42,
                'criterion':trial.suggest_categorical('criterion', ['gini','entropy'])
                 }
    #cross validation using StratifiedKfold
    
    fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=69)
    i = 0
    for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        
        rc = RandomForestClassifier(**param_grid)
        
        rc.fit(xtrain,ytrain)
        
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        
        y_pred = rc.predict(frature_extractor_test)
        i+=1
        
        test_score.append(precision_score(y_pred,test_labels))
        
        #returning mean score
    return np.mean(test_score)

In [19]:
study = optuna.create_study(direction='maximize',sampler=TPESampler(), study_name='Random_forset_optuna')
study.optimize(objective, n_trials = 100)

[I 2022-12-04 08:39:12,972] A new study created in memory with name: Random_forset_optuna


For Split 0
train accuracy:0.8490783410138248
test accuracy:0.8709677419354839
For Split 1
train accuracy:0.8513824884792627
test accuracy:0.8387096774193549
For Split 2
train accuracy:0.8490783410138248
test accuracy:0.8248847926267281
For Split 3
train accuracy:0.8548387096774194
test accuracy:0.7695852534562212
For Split 4


[I 2022-12-04 08:39:14,654] Trial 0 finished with value: 0.19480519480519481 and parameters: {'n_estimators': 84, 'max_depth': 1, 'criterion': 'entropy'}. Best is trial 0 with value: 0.19480519480519481.


train accuracy:0.8582949308755761
test accuracy:0.8433179723502304
For Split 0
train accuracy:0.9861751152073732
test accuracy:0.9308755760368663
For Split 1
train accuracy:0.9919354838709677
test accuracy:0.8847926267281107
For Split 2
train accuracy:0.9838709677419355
test accuracy:0.8709677419354839
For Split 3
train accuracy:0.988479262672811
test accuracy:0.8617511520737328
For Split 4


[I 2022-12-04 08:39:19,414] Trial 1 finished with value: 0.5982683982683983 and parameters: {'n_estimators': 92, 'max_depth': 6, 'criterion': 'entropy'}. Best is trial 1 with value: 0.5982683982683983.
[I 2022-12-04 08:39:19,538] Trial 2 finished with value: 0.16017316017316016 and parameters: {'n_estimators': 3, 'max_depth': 1, 'criterion': 'gini'}. Best is trial 1 with value: 0.5982683982683983.


...

train accuracy:0.8928571428571429
test accuracy:0.8940092165898618
For Split 0
train accuracy:0.9735023041474654
test accuracy:0.9124423963133641
For Split 1
train accuracy:0.9758064516129032
test accuracy:0.8709677419354839
For Split 2
train accuracy:0.978110599078341
test accuracy:0.880184331797235
For Split 3
train accuracy:0.9769585253456221
test accuracy:0.8617511520737328
For Split 4


[I 2022-12-04 08:44:08,517] Trial 98 finished with value: 0.7541125541125541 and parameters: {'n_estimators': 87, 'max_depth': 5, 'criterion': 'gini'}. Best is trial 74 with value: 0.7610389610389611.


train accuracy:0.9723502304147466
test accuracy:0.9170506912442397
For Split 0
train accuracy:0.9447004608294931
test accuracy:0.9170506912442397
For Split 1
train accuracy:0.9550691244239631
test accuracy:0.8709677419354839
For Split 2
train accuracy:0.9527649769585254
test accuracy:0.8894009216589862
For Split 3
train accuracy:0.9516129032258065
test accuracy:0.8617511520737328
For Split 4


[I 2022-12-04 08:44:11,875] Trial 99 finished with value: 0.6692640692640692 and parameters: {'n_estimators': 94, 'max_depth': 4, 'criterion': 'gini'}. Best is trial 74 with value: 0.7610389610389611.


train accuracy:0.9550691244239631
test accuracy:0.9124423963133641


In [20]:
print("Number of trails : {}".format(len(study.trials)))
print("Best parameters :")
print(study.best_params)
params = study.best_params

Number of trails : 100
Best parameters :
{'n_estimators': 93, 'max_depth': 5, 'criterion': 'gini'}


In [21]:
preds = []
acc = 0
prc = 0 
rcf = 0 
f1s = 0 
fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
i = 1
for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        rc = RandomForestClassifier(**params)
        rc.fit(xtrain,ytrain)
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        y_pred = rc.predict(frature_extractor_test)
        preds.append(y_pred)
        acc+=(accuracy_score(y_pred,test_labels))
        prc+=(precision_score(y_pred,test_labels))
        rcf+=(recall_score(y_pred,test_labels))
        f1s+=(f1_score(y_pred,test_labels))
        i+=1
preds = np.mean(np.column_stack(preds), axis=1)
new_preds = []
for x in preds:
    if x>=0.:
        new_preds.append(1)
    else:
        new_preds.append(0)
print(acc/5)
print(prc/5)
print(rcf/5)
print(f1s/5)

For Split 1
train accuracy:0.9723502304147466
test accuracy:0.8847926267281107
For Split 2
train accuracy:0.9758064516129032
test accuracy:0.9032258064516129
For Split 3
train accuracy:0.9792626728110599
test accuracy:0.8571428571428571
For Split 4
train accuracy:0.9804147465437788
test accuracy:0.8940092165898618
For Split 5
train accuracy:0.9804147465437788
test accuracy:0.8940092165898618
0.8194285714285714
0.6311688311688312
0.9387835646803975
0.7538903725933966
